In [1]:
def load_label_encoders(pkl_path: str) -> dict:
    try:
        with open(pkl_path, 'rb') as f:
            label_encoders = pickle.load(f)
        return label_encoders
    except Exception as e:
        return {}

In [2]:
load_label_encoders(r"C:\Users\shari\PycharmProjects\MegaDendrologAI\ML\Classification\results\saved_models\label_encoders.pkl")

{}

In [13]:
import torch
import torch.nn as nn
from tqdm import tqdm
import torch.optim as optim
from torch.utils.data import DataLoader
import json
from PIL import Image
from pathlib import Path
from torch.utils.data import Dataset
from sklearn.preprocessing import LabelEncoder
from typing import List, Dict
import pickle

class BaseDataset(Dataset):
    def __init__(self):
        self.samples = []
        self.label_encoders = {}

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, index):
        image_path, labels_dict = self.samples[index]
        image_encoded = Image.open(image_path).convert('RGB')
        return image_encoded, labels_dict

class ImageDatasetJson(BaseDataset):
    def __init__(
        self, 
        json_path: Path, 
        image_filepath_field: str, 
        target_fields: List[str],
        label_encoders: Dict[str, LabelEncoder] = None
    ):
        self.json_path = json_path
        self.image_filepath_field = image_filepath_field
        self.target_fields = target_fields
        self.samples = []
        
        with open(self.json_path, 'r', encoding='utf-8') as f:
            images_dict = json.load(f)
        
        field_values = {field: [] for field in self.target_fields}
        for image_classified in images_dict:
            for field in self.target_fields:
                field_values[field].append(image_classified[field])
        
        if label_encoders is None:
            self.label_encoders = {}
            for field in self.target_fields:
                le = LabelEncoder()
                le.fit(field_values[field])
                self.label_encoders[field] = le
        else:
            self.label_encoders = label_encoders
        
        for image_classified in images_dict:
            image_filepath = image_classified[self.image_filepath_field]
            
            labels_dict = {}
            for field in self.target_fields:
                value = image_classified[field]
                encoded_value = self.label_encoders[field].transform([value])[0]
                labels_dict[field] = encoded_value
            
            self.samples.append((image_filepath, labels_dict))
    
    def save_label_encoders(self, filepath: Path):
        """Сохраняет LabelEncoder'ы в виде простого JSON с маппингами"""
        
        label_encoders_dict = {}
        
        for field_name, encoder in self.label_encoders.items():
            label_encoders_dict[field_name] = {
                str(cls): int(idx) for idx, cls in enumerate(encoder.classes_)
            }
        
        with open(filepath, 'w', encoding='utf-8') as f:
            json.dump(label_encoders_dict, f, indent=2, ensure_ascii=False)

import torch
from PIL import Image, ImageFile
from typing import List, Tuple, Any, Callable, Dict


class ImageCollator:
    def __init__(
        self,
        processor,
        task_names: List[str]
    ):
        self.processor = processor
        self.task_names = task_names

    def __call__(self, batch):
        images, labels_dict = zip(*batch)
        inputs = self.processor(images, return_tensors="pt")
        
        labels = {}
        for task_name in self.task_names:
            task_labels = [label_dict[task_name] for label_dict in labels_dict]
            labels[task_name] = torch.tensor(task_labels)
        
        inputs["labels"] = labels
        return inputs
    

from pathlib import Path
from transformers import AutoImageProcessor


class TrainConfigs:
    class TreeClassificationModelWithMultiHeadMLP:
        MODEL_NAME = 'microsoft/resnet-18'
        TRAIN_JSON_FILEPATH = Path(r"C:\Users\shari\PycharmProjects\MegaDendrologAI\result.json")
        IMAGE_PROCESSOR = AutoImageProcessor.from_pretrained(
            MODEL_NAME, use_fast=True
        )
        IMAGE_JSON_FIELD = "image"
        TARGET_JSON_FIELD = {
            "tree_type": 16, # number of unique trees based on the QWEN reports
            "has_hollow": 2,
            "has_cracks": 2,
            # "injuries": 30, # number of unique injuries based on the QWEN reports
            "has_fruits_or_flowers": 2,
        }
        PATH_TO_SAVE_MODEL = Path('ML/Classification/results/saved_models/hollow_classification_small.pth')

import torch
import torch.nn as nn
from typing import Dict, List, Union, Optional

class ResNetWrapper(nn.Module):
    def __init__(
        self, 
        resnet_model: str,
        hidden_size: int = 256,
        num_output_features: Union[int, Dict[str, int]] = 2,
        num_hidden_layers: int = 1,
        dropout: float = 0.1,
        activation: str = "relu",
        freeze_resnet: bool = False,
    ):
        super(ResNetWrapper, self).__init__()
        
        self.resnet = ResNetBackbone(model_name=resnet_model)
        
        with torch.no_grad():
            dummy_input = torch.randn(1, 3, 224, 224)
            dummy_output = self.resnet(dummy_input)
            dummy_output = dummy_output.view(dummy_output.size(0), -1)
            self.feature_size = dummy_output.shape[-1]
        
        if freeze_resnet:
            for param in self.resnet.parameters():
                param.requires_grad = False
        
        self.task_names = []
        self.output_layers = nn.ModuleDict()
        
        if isinstance(num_output_features, int):
            self.task_names = ['main']
            self.output_layers['main'] = self._create_mlp(
                input_size=self.feature_size,
                hidden_size=hidden_size,
                output_size=num_output_features,
                num_hidden_layers=num_hidden_layers,
                dropout=dropout,
                activation=activation
            )
        else:
            self.task_names = list(num_output_features.keys())
            for task_name, num_classes in num_output_features.items():
                self.output_layers[task_name] = self._create_mlp(
                    input_size=self.feature_size,
                    hidden_size=hidden_size,
                    output_size=num_classes,
                    num_hidden_layers=num_hidden_layers,
                    dropout=dropout,
                    activation=activation
                )
    
    def _create_mlp(
        self, 
        input_size: int,
        hidden_size: int,
        output_size: int,
        num_hidden_layers: int,
        dropout: float,
        activation: str
    ) -> nn.Sequential:
        layers = []
        current_input_size = input_size
        
        for i in range(num_hidden_layers):
            layers.extend([
                nn.Linear(current_input_size, hidden_size),
                self._get_activation(activation),
                nn.Dropout(dropout)
            ])
            current_input_size = hidden_size
        
        layers.append(nn.Linear(current_input_size, output_size))
        
        return nn.Sequential(*layers)
    
    def _get_activation(self, activation: str) -> nn.Module:
        activations = {
            "relu": nn.ReLU(),
            "leaky_relu": nn.LeakyReLU(0.1),
            "gelu": nn.GELU(),
            "tanh": nn.Tanh(),
            "sigmoid": nn.Sigmoid()
        }
        return activations.get(activation, nn.ReLU())
    
    def forward(self, x: torch.Tensor) -> Union[torch.Tensor, Dict[str, torch.Tensor]]:
        features = self.resnet(x)
        if features.dim() == 4:  # [batch, channels, height, width]
            features = features.view(features.size(0), -1)

        if len(self.task_names) == 1:
            return self.output_layers[self.task_names[0]](features)
        else:
            return {task_name: self.output_layers[task_name](features) 
                   for task_name in self.task_names}
    
    def unfreeze_resnet(self, unfreeze: bool = True):
        for param in self.resnet.parameters():
            param.requires_grad = unfreeze
import json
from pathlib import Path

CURRENT_CONFIG = TrainConfigs.TreeClassificationModelWithMultiHeadMLP


task_names = list(CURRENT_CONFIG.TARGET_JSON_FIELD.keys())
num_classes_per_task = CURRENT_CONFIG.TARGET_JSON_FIELD


train_dataset = ImageDatasetJson(
    CURRENT_CONFIG.TRAIN_JSON_FILEPATH,
    CURRENT_CONFIG.IMAGE_JSON_FIELD,
    target_fields=task_names
)


encoders_path = CURRENT_CONFIG.PATH_TO_SAVE_MODEL.parent / "label_encoders.pkl"
train_dataset.save_label_encoders(encoders_path)

In [16]:
le = load_label_encoders(r"C:\Users\shari\PycharmProjects\MegaDendrologAI\ML\Classification\results\saved_models\label_encoders.pkl")

c:\Users\shari\PycharmProjects\MegaDendrologAI\.venv\lib\site-packages\sklearn\base.py:442: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.7.1 when using version 1.7.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [23]:
le_dict = {}
for name, encoder in le.items():
    le_dict[name] = {}
    for key, value in enumerate(encoder.classes_):
        le_dict[name][int(key)] = str(value)
le_dict

{'tree_type': {0: 'Береза',
  1: 'Вяз',
  2: 'Дуб',
  3: 'Ель',
  4: 'Ива',
  5: 'Каштан',
  6: 'Клен остролистный',
  7: 'Клен ясенелистный',
  8: 'Липа',
  9: 'Лиственница',
  10: 'Осина',
  11: 'Рябина',
  12: 'Сосна',
  13: 'Туя',
  14: 'Ясень',
  15: 'неопределено'},
 'has_hollow': {0: '0', 1: '1'},
 'has_cracks': {0: '0', 1: '1'},
 'has_fruits_or_flowers': {0: '0', 1: '1'}}